In [4]:
import numpy as np
import pandas as pd
from rdkit.Chem import MolStandardize


RDKit WARNING: [16:10:24] Enabling RDKit 2019.09.3 jupyter extensions


In [5]:
def mol2alt_sentence(mol, radius):
    """Same as mol2sentence() expect it only returns the alternating sentence
    Calculates ECFP (Morgan fingerprint) and returns identifiers of substructures as 'sentence' (string).
    Returns a tuple with 1) a list with sentence for each radius and 2) a sentence with identifiers from all radii
    combined.
    NOTE: Words are ALWAYS reordered according to atom order in the input mol object.
    NOTE: Due to the way how Morgan FPs are generated, number of identifiers at each radius is smaller

    Parameters
    ----------
    mol : rdkit.Chem.rdchem.Mol
    radius : float
        Fingerprint radius

    Returns
    -------
    list
        alternating sentence
    combined
    """
    radii = list(range(int(radius) + 1))
    info = {}
    _ = AllChem.GetMorganFingerprint(mol, radius, bitInfo=info)  # info: dictionary identifier, atom_idx, radius

    mol_atoms = [a.GetIdx() for a in mol.GetAtoms()]

    #     print(mol_atoms)
    dict_atoms = {x: {r: None for r in radii} for x in mol_atoms}

    for element in info:
        for atom_idx, radius_at in info[element]:
            dict_atoms[atom_idx][radius_at] = element  # {atom number: {fp radius: identifier}}

    # merge identifiers alternating radius to sentence: atom 0 radius0, atom 0 radius 1, etc.
    identifiers_alt = []
    for atom in dict_atoms:  # iterate over atoms
        for r in radii:  # iterate over radii
            identifiers_alt.append(dict_atoms[atom][r])

    alternating_sentence = map(str, [x for x in identifiers_alt if x])

    return list(alternating_sentence)

In [6]:
allowedAtomsDict = {
    'H' : 1,'h' : 0,
    'B' : 5,'b' : 0,
    'C' : 6,'c' : 0,
    'N' : 7,'n' : 0,
    'O' : 8,'o' : 0,
    'F' : 9,'f' : 0,
    'P' : 15,'p': 0,
    'S' : 16,'s': 0,
    'Cl': 17,'Br' : 35
}


In [7]:
word = "AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZzBrCl"
def isValidCharacter(c):
    if c not in word or (c in word and c in "HhBbCcNnOoFfPpSsClBr"):
        return True
    return False

def isValidSmiles(smiles,atom_weight = 600,heavy_atom_count = 50):
    '''
        1. smiles能够被rdkit包处理
        2. smiles只包含特定元素
        3. smiles原子权重
    '''
    t_weight = 0
    heavyAtomCount = 0
    left = -len(smiles)-1
    right = -1
    idx = -1
    while True:
        if idx <= left:
            break
        c = smiles[idx]
        if smiles[idx] == 'r' or smiles[idx] == 'l' :
            c = (smiles[idx-1] if idx -1 > right else "#") + c
            idx = idx - 1
        idx = idx - 1
        if isValidCharacter(c) == True:
            if c in allowedAtomsDict.keys():
                t_weight = t_weight + int(allowedAtomsDict[c])
                heavyAtomCount = heavyAtomCount + (1 if int(allowedAtomsDict[c]) > 1 else 0)
        else:
            return False
#     print(type(t_weight),ttype(heavy_atom_count))
    return  True if t_weight >= 3 and t_weight <= atom_weight and heavyAtomCount <= heavy_atom_count else False

In [8]:
def standardizeAndcanonical(smi):
    lfc = MolStandardize.fragment.LargestFragmentChooser()
    # standardize
    mol = Chem.MolFromSmiles(smi)
    mol2 = lfc.choose(mol)
    smi2 = Chem.MolToSmiles(mol2)
    #     print(smi2)
    #     # canonical
    #     can_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi2))
    # #     print(can_smi)
    #     print(can_smi == smi2)
    return smi2

In [9]:
import csv
import pandas as pd
import random
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import pandas as pd


In [10]:
# 预先加载数据集
df = pd.read_csv('raw/SAMPL.csv',sep=',')
# print(df)
smiles_list = []
# line = line.rstrip().split(',')
# smiles = str(line[8])
# val = float(line[7]
for _,smlAndlabel in df.iterrows():
    smiles, label = smlAndlabel[1], smlAndlabel[2]
    smiles_list.append([smiles,label])
# print(smiles_list)

In [11]:
def generate_scaffold_file( smiles_list, num=0,):
    new_smiles_list = []
    for i in range(len(smiles_list)):
        smiles = smiles_list[i][0]
        label = smiles_list[i][1]
        smi = standardizeAndcanonical(smiles)
        if isValidSmiles(smi) == True:
            t = Chem.MolFromSmiles(smi)
            if t != None:
                sentence = mol2alt_sentence(t,1)
#                     print(sentence[0] == 'none')
                if len(sentence) % 2 == 0:
                        new_smiles_list.append([smiles, label])
    idx = [i for i in range(len(new_smiles_list))]
    random.shuffle(idx)
    print(idx)
    idx_train, idx_validation, idx_test = idx[0:int(len(idx)*0.8)],idx[int(len(idx)*0.8):int(len(idx)*0.9)]\
    ,idx[int(len(idx)*0.9):]
    print(len(idx_train), len(idx_validation), len(idx_test))
    with open("train_" + str(num) + ".txt","w") as f:
        for id in idx_train:
            smiles = smiles_list[id][0]
            label = smiles_list[id][1]
            f.write(smiles+","+str(label)+"\n")
            
    with open("validation_" + str(num) + ".txt","w") as f:
        for id in idx_validation:
            smiles = smiles_list[id][0]
            label = smiles_list[id][1]
            f.write(smiles+","+str(label)+"\n")
            
    with open("test_" + str(num) + ".txt","w") as f:
        for id in idx_test:
            smiles = smiles_list[id][0]
            label = smiles_list[id][1]
            f.write(smiles+","+str(label)+"\n")
    

In [12]:
generate_scaffold_file(smiles_list,0)
generate_scaffold_file(smiles_list,1)
generate_scaffold_file(smiles_list,2)
generate_scaffold_file(smiles_list,3)
generate_scaffold_file(smiles_list,4)
generate_scaffold_file(smiles_list,5)

[157, 310, 470, 503, 481, 5, 170, 450, 42, 454, 139, 343, 559, 240, 339, 199, 362, 138, 250, 269, 188, 377, 167, 356, 65, 335, 308, 433, 20, 24, 31, 506, 80, 7, 303, 168, 576, 369, 86, 480, 371, 497, 562, 134, 10, 200, 53, 242, 538, 338, 103, 136, 557, 182, 41, 478, 38, 46, 311, 441, 388, 424, 568, 233, 128, 166, 451, 176, 161, 372, 404, 510, 271, 225, 465, 253, 314, 430, 280, 296, 530, 408, 54, 385, 432, 266, 6, 21, 143, 521, 267, 302, 390, 115, 291, 405, 259, 446, 154, 75, 301, 209, 226, 36, 151, 306, 555, 229, 486, 286, 230, 466, 26, 106, 320, 257, 407, 9, 355, 489, 457, 70, 360, 438, 548, 144, 194, 231, 354, 288, 152, 212, 104, 406, 549, 29, 341, 290, 358, 156, 191, 228, 546, 217, 471, 520, 219, 512, 165, 349, 392, 205, 418, 281, 118, 542, 239, 366, 498, 431, 19, 426, 39, 91, 220, 496, 221, 276, 55, 370, 13, 573, 389, 126, 61, 380, 350, 459, 282, 397, 558, 416, 342, 304, 551, 111, 248, 109, 44, 493, 516, 119, 519, 347, 487, 373, 312, 232, 18, 285, 140, 71, 494, 439, 94, 391, 394, 5

[519, 295, 92, 96, 269, 378, 530, 330, 563, 297, 396, 307, 428, 34, 515, 375, 125, 43, 78, 482, 95, 8, 557, 225, 543, 315, 114, 488, 381, 304, 86, 333, 188, 451, 395, 538, 210, 230, 522, 229, 248, 119, 321, 20, 271, 548, 183, 453, 460, 459, 516, 185, 441, 46, 179, 365, 477, 72, 442, 535, 371, 62, 144, 4, 280, 427, 331, 346, 394, 53, 71, 501, 13, 495, 447, 291, 416, 222, 160, 141, 130, 148, 576, 491, 203, 360, 327, 404, 508, 425, 218, 476, 319, 443, 398, 308, 168, 88, 260, 267, 118, 366, 577, 212, 294, 525, 121, 65, 311, 528, 11, 253, 357, 89, 235, 154, 58, 21, 296, 471, 129, 48, 422, 177, 306, 223, 439, 533, 455, 424, 272, 277, 167, 73, 517, 318, 12, 147, 211, 347, 152, 134, 25, 240, 433, 487, 195, 83, 364, 344, 423, 537, 506, 462, 151, 133, 181, 335, 237, 300, 232, 155, 234, 7, 165, 454, 362, 470, 349, 483, 138, 358, 520, 50, 257, 209, 549, 570, 303, 499, 340, 122, 541, 259, 448, 273, 292, 171, 9, 127, 417, 374, 529, 44, 312, 386, 310, 220, 492, 401, 219, 571, 469, 270, 275, 107, 518,